# Collaborative Filtering Recommendations using Original Values

In [1]:
import pandas as pd
import numpy as np
import implicit

from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix

In [2]:
train = pd.read_csv("data\\train-plays.csv")
test = pd.read_csv("data\\test-plays.csv")

## Format training data

In [3]:
game_user = train.pivot(index="game_name", columns="user_id", values="amount")

In [4]:
game_user = game_user.fillna(0)

In [5]:
# create user and game incodings
user_coding = pd.DataFrame({"original":game_user.columns, "coded":np.arange(game_user.columns.size)})
game_coding = pd.DataFrame({"original":game_user.index, "coded":np.arange(game_user.index.size)})

In [6]:
game_user.columns = user_coding['coded']
game_user.index = game_coding['coded']

In [25]:
game_user_sparse = csr_matrix(game_user)

In [22]:
user_game = game_user.T

In [23]:
user_game_sparse = csr_matrix(user_game)

# Alternating Least Squares

In [14]:
als = implicit.als.AlternatingLeastSquares(128, use_gpu=True)

In [26]:
als.fit(game_user_sparse)

100%|██████████| 15/15 [00:00<00:00, 29.59it/s]


In [30]:
pd.DataFrame(als.recommend(0, user_game_sparse)).merge(game_coding, left_on=0, right_on="coded")

,0,1,original,coded
0,361,0.765361,Call of Duty Black Ops,361
1,370,0.567266,Call of Duty Modern Warfare 2,370
2,372,0.395861,Call of Duty Modern Warfare 3,372
3,1719,0.382153,Portal,1719
4,876,0.372286,Fallout New Vegas,876
5,1426,0.371800,Medieval II Total War,1426
6,1903,0.340836,Rome Total War,1903
7,227,0.312294,Battlefield Bad Company 2,227
8,2286,0.306539,Team Fortress 2,2286
9,697,0.300643,Dota 2,697


In [36]:
test[test['user_id'] == 76767]

,user_id,game_name,amount,norm_amount
1029,76767,Call of Duty Black Ops,22.0,0.057692
2049,76767,Call of Duty Modern Warfare 2,65.0,0.175824
6112,76767,Thief Deadly Shadows,3.4,0.006593
10243,76767,Call of Duty Modern Warfare 3,15.9,0.040934


In [39]:
user_coding

,original,coded
0,76767,0
1,86540,1
2,229911,2
3,298950,3
4,554278,4
...,...,...
2037,290542222,2037
2038,293042025,2038
2039,294620986,2039
2040,296283194,2040


In [37]:
train[train['user_id'] == 76767]

,user_id,game_name,amount,norm_amount
475,76767,Total War ATTILA,207.0,0.565934
5858,76767,Call of Duty Black Ops - Multiplayer,12.5,0.031593
6734,76767,Banished,24.0,0.063187
7792,76767,Counter-Strike Source,25.0,0.065934
10295,76767,Counter-Strike,365.0,1.000000
11550,76767,Counter-Strike Global Offensive,3.5,0.006868
12243,76767,Call of Duty Modern Warfare 3 - Multiplayer,9.7,0.023901
15523,76767,Call of Duty Modern Warfare 2 - Multiplayer,165.0,0.450549
16121,76767,Portal 2,15.0,0.038462
18993,76767,The Stanley Parable,1.8,0.002198
